In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Recurrence relations ##

Most of the time we'll be interested in evaluating sums of coefficients times special functions, but it is sometimes useful to have the special functions themselves.  The best way to handle functions with recurrence relations is the use ... recurrence relations.

Let's consider two examples.

### Legendre polynomials ###

For Legendre polynomials everything is easy.  Recall $P_0(x)=1$, $P_1(x)=x$ and
$$
  P_{\ell+1}(x) = \frac{2\ell+1}{\ell+1}x\,P_\ell(x) - \frac{\ell}{\ell+1}\,P_{\ell-1}(x)
$$
This allow us to compute any $P_\ell(x)$ that we like, for example $\ell=10$.

In [ ]:
# Example of recurrence evaluation for Legendre polynomial.
xx = 0.5
ell= 10
p0 = 1.0
p1 = xx
pl = 0.0
# Do this explicitly for clarity.  Note it wouldn't be hard to
# keep the individual P_ell(x) as we compute them (in a table
# or array) if we wanted.
for lval in range(1,ell):
    pl = (2.*lval+1.)/(lval+1.)*xx*p1 - lval/(lval+1.)*p0
    p0 = p1
    p1 = pl
print("LegendreP_{:d}({:f})={:f}".format(ell,xx,pl))

This can be compared to the "exact" value for the polynomial of -0.188229.

**Note** that this recurrence is "upwardly stable", so errors die off as we go up the hierarchy (which is very convenient). In the introduction to Abramowitz and Stegun there is a useful list of upwardly and downwardly stable recurrence relations.

### Bessel functions ###

An example of a function with an upwardly unstable recurrence relation is a Bessel function.
For these we have
$$
  J_0(x) = 1 - \frac{x^2}{4} + \frac{x^4}{64} - \frac{x^6}{2304} + \cdots
  \quad , \quad
  J_1(x) = \frac{x}{2} - \frac{x^3}{16} + \frac{x^5}{384} - \frac{x^7}{18432} + \cdots
$$
or
$$
  J_0(x) = \frac{1}{\pi}\int_0^\pi dt\ \cos\left(x\sin t\right)
  \quad , \quad
  J_1(x) = \frac{1}{\pi}\int_0^\pi dt\ \cos\left(t-x\sin t\right)
$$
so we can again start a recurrence and move up:
$$
  J_{n+1}(x) = \frac{2n}{x} J_{n}(x) - J_{n-1}(x)
$$

In [ ]:
# Consider doing an upwards recurrence on the Bessel functions for x=1/2.
# This won't work, because the relation is unstable in the upwards direction.
xx = 0.5
x2 = xx**2
NN = 10
J0 = 1 + x2/4*(-1 + x2/16*(1-x2/36.))
J1 = xx/2*(1+x2/8*(-1+x2/24*(1-x2/48)))
Jn = 0.0
# Now go up the recurrence.
for n in range(1,NN):
    Jn = (2.*n)/xx * J1 - J0
    J0 = J1
    J1 = Jn
print("J_{:d}({:f})={:f}".format(NN,xx,Jn))

This can be compared to the correct value $3\times 10^{-13}$ (which the series expansion would tell you very quickly).  Not a good approximation!

Of course we wouldn't do $J_n(x)$ by recurrence if $x\ll 1$ since the series would converge so quickly.  In fact it tends to converge pretty quickly as long as $x<n$.  And for $x\gg n$ it turns out the recurrence above is pretty stable.  But what if $x\sim n$ so the series converges too slowly to be useful, the recurrence is unstable (and we can't get clever and use the expansion in $1/x$ either)?

If you wanted to evaluate $J_n$ through recurrence, the trick is to start at a very high value and then work down to $n$.  The relation is stable downwards.  How do we start?  Well, since the error is going to die exponentially we can start with $J_{N+1}=0$ and $J_{N}=1$, work down to $J_0$ and then multiply everything in our sequence by the true $J_0$ at the end (we can choose any helpful starting value for $J_{N}$).  This is known as Miller's method.

In [ ]:
# Example of doing Bessel recurrence downwards to get J_{10}(10), for which
# the series would take a *long* time to converge and the high-x asymptote is
# also not particularly accurate.
xx = 10.0
NN = 10
J0 = -0.245936
Jn = np.zeros(int(2*xx+1))  # Choosing Nmax ~ 2x is a reasonable guess.
Jn[-2] = 1.0e-10            # Could have chosen 1 as well.
# Now go down the recurrence.
for n in range(Jn.size-2,0,-1):
    Jn[n-1] = (2*n)/xx * Jn[n] - Jn[n+1]
Jn   *= J0/Jn[0]
print("J_{:d}({:f})={:f}".format(NN,xx,Jn[NN]))

This can be compared to the true value $J_{10}(10)=0.207486$.  Not bad!  Of course we still need an approximation for $J_0(x)$ that we can trust for all $x$.  We can use power series for large and small $x$ but people normally use a pair of Pade approximants or rational function approximations valid for $x\le x_{\rm cut}$ and $x\ge x_{\rm cut}$.  These have been worked out historically by numerous authors (famous and otherwise) and can be found in lots of places.

Ok, but what if we don't want to keep tables of special functions but just want to evaluate a sum with some known coefficients?  For this we tend to use Clenshaw's method.

## Clenshaw's method ##

Sometimes it is useful to have the values of the special functions themselves, but more frequently we need to evaluate sums of these special functions times coefficients.  Imagine we want to evaluate $\sum_k c_k f_k(x)$, where $f_k(x)$ are functions (arising from second order differential equations) which obey a recurrence relation of the general form
$$
  f_{k+1}(x)=\alpha_k(x) f_k(x)+\beta_k(x) f_{k-1}(x)
$$
Direct substitution shows
$$
  \sum_{k=0}^N c_kf_k(x)=\beta_1(x)f_0(x)y_2+f_1(x)y_1+f_0(x)c_0
  \quad {\rm where}\quad
  y_k=\alpha_k(x)y_{k+1}+\beta_{k+1}(x)y_{k+2}+c_k
$$
with $y_{N+2}=y_{N+1}=0$.  To see this consider
\begin{eqnarray}
  \sum_k c_kf_k &=& \cdots \nonumber \\
                &+& \left[ y_3-\alpha_3y_4-\beta_4y_5\right] f_3 \nonumber \\
                &+& \left[ y_2-\alpha_2y_3-\beta_3y_4\right] f_2 \nonumber \\
                &+& \left[ y_1-\alpha_1y_3-\beta_2y_3\right] f_1 \nonumber \\
                &+& \left[ c_0+\beta_1 y_2-\beta_1y_2\right] f_0
\end{eqnarray}
where I have suppressed the $x$-dependence throughout.
Note that the terms involving $y_3$ are $f_3-\alpha_2f_2-\beta_2f_1=0$.
Similarly for the other terms.  The only surviving terms in the sum are
those given above.

The [article on Wikipedia](https://en.wikipedia.org/wiki/Clenshaw_algorithm) uses a slightly different notation.  If
$\phi_{k+1}(x) = \alpha_k(x)\phi_k(x) + \beta_k(x)\phi_{k-1}(x)$ and we define $b_{n+2}(x)=b_{n+1}(x)=0$ then the following iteration:
$$
b_k(x) = a_k + \alpha_k(x) b_{k+1}(x) + \beta_{k+1}(x) b_{k+2}(x)
\quad \Rightarrow\quad
\sum_{k=0}^{n} a_k\phi_k(x) = \left[a_0+\beta_1(x)b_2(x)\right]\phi_0(x)+b_1(x)\phi_1(x)
$$
evaluates the sum.  This translates more directly into pseudo-code and will be used below.

(Occassionally Clenshaw's algorithm can become unstable.  In this case there is an upward variant starting from $y_{-2}=y_{-1}=0$.)

Let us evaluate
$$
\sum_{\ell=1}^L \frac{(2\ell+1)}{\ell(\ell+1)}\ P_\ell(\mu)
$$
for $L=5$, $50$, $500$?  The exact answer turns out to be $\ln 2-1-\ln(1-\mu)$.

Recall
$$
  P_{\ell+1}(\mu) = \frac{2\ell+1}{\ell+1}\mu\,P_\ell(\mu) - \frac{\ell}{\ell+1}\,P_{\ell-1}(\mu)
$$

In [ ]:
# We'll use the Numerical Recipes form and do the iteration explicitly.
def clenshaw_ex1(mu,Lmax):
    b2,b1 = 0.0,0.0   # These stand for b_{k+2} and b_{k+1}
    # Note that a0=infty and the sum starts at ell=1.
    # I'm mixing ell and k here, but hopefully it's still clear.
    for ell in range(Lmax,0,-1):
        ak     =  0 if ell==0 else (2.*ell+1.0)/(ell+1.0)/ell
        alphak = (2.*ell+1.0)/(ell+1.0) * mu
        betak1 =   -(ell+1.0)/(ell+2.0)
        b0     = ak + alphak*b1 + betak1*b2
        b1,b2  = b0,b1 
    P0,P1,beta1= 1.0,mu,-0.5
    return( (0+beta1*b2)*P0+b1*P1 )
    #
def exact_ex1(mu):
    res = np.log(2)-1.0 - np.log(1-mu)
    return(res)

In [ ]:
mu=0.5
print("mu=",mu,", Sum is ",exact_ex1(mu))
print("----")
for Lmax in [5,50,500]:
    ex = exact_ex1(mu)
    ap = clenshaw_ex1(mu,Lmax)
    print("{:4d} {:10.5f} {:10.5f}".format(Lmax,ap,100*(ap-ex)/ex))

__Aside__: The famous [Horner's method](https://en.wikipedia.org/wiki/Horner%27s_method) for evaluating a polynomial (and doing synthetic polynomial division for root finding) can also be cast as a special case of Clenshaw's algorithm.  In this special case
$$
  S(x) = \sum_{k=0}^n a_k x^k \quad , \quad
  \phi_k(x) = x^k = x\phi_{k-1}(x) \Rightarrow \alpha_k=x\ ,\ \beta_k=0
$$
and Clenshaw's algorithm becomes
\begin{eqnarray}
  0   &=& b_{n+2} = b_{n+1} \\
  b_k &=& a_k + x b_{k+1}
\end{eqnarray}
and $S(x)=a_0 + xb_1 = b_0$.

In [ ]:
# Consider a case where we sum over Bessel functions.
# Take a_n=1/n^2 for n>=1.
#
# We need routines for J0 and J1 to get started... here we're going
# to cheat and import them from SciPy, which is silly since if we
# already had them we'd probably use them, but ... in fact J0 and J1
# are pretty easy to find approximations for.
from scipy.special import jv
#
def clenshaw_bessel(x,Nmax):
    b1,b2 = 0.0,0.0
    for n in range(Nmax,0,-1):
        an     = 0 if n==0 else 1.0/n**2
        alphan = 2*n/x
        betan1 = -1.0  # Note is beta_{n+1} but doesn't matter.
        b0     = an + alphan*b1 + betan1*b2
        b1,b2  = b0,b1
    J0,J1,beta1= jv(0,x),jv(1,x),-1.0
    return( (0+beta1*b2)*J0+b1*J1 )
    #
def exact_bessel(x,Nmax):
    nn = range(Nmax+1)
    an = 1.0/(np.array(nn)**2+1e-20)
    an[0]=0.0
    Jn = jv(nn,x)
    return(np.sum(an*Jn))
#
# For x=1 the series converges very quickly, since J_n drops off so quickly
# with n.  Note however this is where our usual application of the recurrence
# relation would have trouble since 2n/x can become quite large.  With Clenshaw
# everything "just works" as long as we keep Nmax "reasonable".  If we push it
# too far the one gets very large b1 and b2 that have to cancel in the last line;
# but if you knew what your functions looked like you wouldn't be trying to
# include J_{20}(1)/20^2 in your sum!  There is a way of trying to detect this
# automatically, see Numerial Recipes.
for Nmax in [5,10,20]:
    print("Sum up to N={:2d} is {:f}, Clenshaw gives {:f}.".\
          format(Nmax,exact_bessel(1.0,Nmax),clenshaw_bessel(1.0,Nmax)))